In [2]:
# Useful libraries
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision

import matplotlib.pyplot as plt
from torchvision import datasets, models, transforms
from torch.utils.data import TensorDataset
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt

from torch.cuda import is_available

In [3]:
class SLR(nn.Module):
    def __init__(self):
        self.name = "SLR"
        super(SLR, self).__init__()
        self.fc1 = nn.Linear(256*6*6, 200)
        self.fc2 = nn.Linear(200, 80)
        self.fc3 = nn.Linear(80,10)
        self.DropOut = nn.Dropout(0.2)
        
    def forward(self,x):
        x = x.view(-1,256*6*6)
        x = self.DropOut(x)
        x = F.relu(self.fc1(x))
        x = self.DropOut(x)
        x = F.relu(self.fc2(x))
        x = self.DropOut(x)
        x = self.fc3(x)
        return x

In [4]:
path = 'dataset/'

train_set_features = torchvision.datasets.DatasetFolder(path + 'TrainFeatures', loader=torch.load, extensions=('.tensor'))
valid_set_features = torchvision.datasets.DatasetFolder(path + 'ValidFeatures', loader=torch.load, extensions=('.tensor'))
test_set_features = torchvision.datasets.DatasetFolder(path + 'TestFeatures', loader=torch.load, extensions=('.tensor'))
print("train set size: ", len(train_set_features))
print("validation set size: ", len(valid_set_features))
print("test set size: ", len(test_set_features))

train set size:  3892
validation set size:  485
test set size:  489


In [5]:
# function to calculate accuracy
def get_accuracy(model, train_loader, valid_loader, train=False):
    if train:
        data = train_loader
    else:
        data = valid_loader

    correct = 0
    total = 0
    for imgs, labels in data:
        
        #enable GPU Usage
        if torch.cuda.is_available():
          imgs = imgs.cuda()
          labels = labels.cuda()

        output = model(imgs)
        
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

def get_test_accuracy(model, test_loader):
    data = test_loader
    correct = 0
    total = 0
    
    for imgs, labels in data:
        if torch.cuda.is_available():
            imgs = imgs.cuda()
            labels = labels.cuda()
            
        output = model(imgs)
    
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct/total

In [6]:
from torch.cuda import is_available
def train(model, train_set, valid_set, batch_size=50, learning_rate=0.001,beta1=0.9, beta2=0.999, eps=1e-8, num_epochs=1):
  
  print("Loading data...")
  #load data
  train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle= True)
  valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, shuffle= True)
  
  #Cross Entropy Loss and SGD optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(beta1,beta2), eps=eps)

  iters, losses, train_acc, valid_acc = [],[],[],[]
  print("Starting training...")
  #training
  n = 0 # of iterations
  for epoch in range(num_epochs):
    for imgs, labels in iter(train_loader):

      #enable GPU
      if torch.cuda.is_available():
        imgs = imgs.cuda()
        labels = labels.cuda()

      out = model(imgs)  #forward pass
      loss = criterion(out, labels) #compute total loss
      loss.backward() #backward pass and parameter update
      optimizer.step() #make the updates for each parameter
      optimizer.zero_grad() #clean up

      #save training information
      iters.append(n)
      losses.append(float(loss)/batch_size) #compute "average" loss
      train_acc.append(get_accuracy(model,train_loader,valid_loader,train=True))
      valid_acc.append(get_accuracy(model,train_loader,valid_loader,train=False))
      if n%10 == 0:
        print("Iter number ",n, " train accuracy: ",train_acc[n], " valid accuracy: ",valid_acc[n])
      n+=1
      
    #save model

    model_path = "TrainingModel/model_{0}_bs{1}_lr{2}_epoch{3}".format(model.name, batch_size, learning_rate, epoch)
    torch.save(model.state_dict(), model_path)
    print("epoch: ", epoch)

  plt.title("Training Curve")
  plt.plot(iters, losses, label="Train")
  plt.xlabel("Iterations")
  plt.ylabel("Loss")
  plt.show()

  plt.title("Training Curve")
  plt.plot(iters, train_acc, label="Train")
  plt.plot(iters, valid_acc, label="Validation")
  plt.xlabel("Iterations")
  plt.ylabel("Training Accuracy")
  plt.legend(loc='best')
  plt.show()

  print("Final Training Accuracy: {}".format(train_acc[-1]))
  print("Final Validation Accuracy: {}".format(valid_acc[-1]))

In [7]:
model = SLR()
if torch.cuda.is_available():
    model.cuda()
    print("train on GPU...")

train(model,train_set_features,valid_set_features, batch_size=32, learning_rate=0.001, num_epochs=100)

train on GPU...
Loading data...
Starting training...
Iter number  0  train accuracy:  0.10149023638232271  valid accuracy:  0.09690721649484536
Iter number  10  train accuracy:  0.11279547790339157  valid accuracy:  0.12371134020618557
Iter number  20  train accuracy:  0.11253854059609456  valid accuracy:  0.10721649484536082
Iter number  30  train accuracy:  0.13129496402877697  valid accuracy:  0.13814432989690723
Iter number  40  train accuracy:  0.14362795477903392  valid accuracy:  0.12989690721649486
Iter number  50  train accuracy:  0.13206577595066804  valid accuracy:  0.12371134020618557
Iter number  60  train accuracy:  0.13540596094552929  valid accuracy:  0.1402061855670103
Iter number  70  train accuracy:  0.1485097636176773  valid accuracy:  0.14845360824742268
Iter number  80  train accuracy:  0.14028776978417265  valid accuracy:  0.12783505154639174
Iter number  90  train accuracy:  0.17317574511819117  valid accuracy:  0.1422680412371134
Iter number  100  train accurac

KeyboardInterrupt: 